# Figure 6 - Layout

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *
from visualization import *

from scipy.stats import zscore

# Loading data for figure

In [ ]:
planes = np.load('../Results/figure6_imaging_planes.npy')
timeseries = np.load('../Results/figure6_timeseries.npy')

d_values, FC_values = np.load('../Results/figure6_scatter_distance.npy')
FC_matrix = np.load('../Results/figure6_tectum_correlations.npy')
density = compute_scatter_density(d_values, FC_values)

tectal_gradients = np.load('../Results/figure6_tectal_gradients.npy')
tectal_eigenmodes = np.load('../Results/figure6_tectal_eigenmodes.npy')
mode_similarity = np.load('../Results/figure6_similarity_matrix.npy')
mapping = np.load('../Results/figure6_mode_mapping.npy')

# Making figure layout

In [ ]:
cmap = make_cmap([[0, 0, 0], [0, 1, 0], [1, 1, 1]], [0, 0.5, 1.0])

In [ ]:
fig = PaperFigure(figsize=(7, 7))

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding axes ----------------------------------------------------------------------------

fig.add_axes('tectum', (3, 0), 3 * 0.7, 1.5)
fig.add_axes('timeseries', (5.8, 0), 1.5, 3.5)

fig.add_axes('correlations', (1.5, 2), 1.5, 1.5)
fig.add_axes('scatter', ((3 + 3 * 0.7) - 1.5, 2), 1.5, 1.5)

fig.add_axes('mapping', (5.5, 4), 1.5, 1.5)

N_modes = 7
w = 5 / N_modes
for i in range(N_modes):
    fig.add_axes(f'eigenmode{i}', (i * w, 4), w, w)
    fig.add_axes(f'gradient{i}', (i * w, 4 + w), w, w)

fig.set_line_thickness(0.5)

ax = fig.axes['tectum']
ax.imshow(planes[5], aspect='auto', cmap='gray', vmin=1000, vmax=15000)
ax.set_xlim([90, 915])
ax.axis('off')

ax = fig.axes['timeseries']
ax.imshow(zscore(timeseries[:, 200:], axis=1), cmap='binary', vmin=0, vmax=1, aspect='auto')
ax.axis('off')

cmap = make_cmap([[1, 1, 1], [1, 0, 0]], [0, 1])
ax = fig.axes['correlations']
ax.imshow(FC_matrix, vmin=-0.25, vmax=0.25, cmap='coolwarm')

ax = fig.axes['scatter']
#ax.scatter(d_values, FC_values, s=0.25, edgecolor=None, alpha=0.1, color='black')
o = np.argsort(density)
v = np.max(density)
ax.scatter(d_values[o], FC_values[o], edgecolor=None, c=density[o], cmap='coolwarm', s=1, alpha=0.1, vmin=-1.25 * v, vmax=v, rasterized=True)
ax.set_xticks([0, 275])
ax.set_xlim([0, 275])
ax.set_ylim([-0.1, 0.6])
ax.set_yticks([0, 0.6])
ax.spines[['top', 'right']].set_visible(False)

ax = fig.axes['mapping']
ax.imshow(np.abs(mode_similarity), cmap='Reds', vmin=0.1, vmax=0.75)
ax.set_xticks([])
ax.set_yticks([])

for i in range(N_modes):
    ax = fig.axes[f'eigenmode{i}']
    ax.imshow(tectal_eigenmodes[i], aspect='auto')
    ax.axis('off')

for i in range(N_modes):
    ax = fig.axes[f'gradient{i}']
    ax.imshow(tectal_gradients[mapping[i]], aspect='auto')
    ax.axis('off')

fig.show()

In [ ]:
fig.save('../Figures/figure6_incomplete.svg')

# Manually written numbers

In [ ]:
mapping

In [ ]:
timeseries.shape